In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df=pd.read_csv('df_1000.csv')

In [3]:
df.shape

(356255, 1022)

In [4]:
for bin_feature in ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

In [21]:
set(df['FLAG_OWN_CAR'])

{0, 1}

In [22]:
set(df['FLAG_OWN_REALTY'])

{0, 1}

In [23]:
df.shape

(356255, 1022)

In [5]:
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [6]:
df, cat_cols = one_hot_encoder(df, nan_as_category=False)

In [7]:
df.shape

(356255, 1141)

In [8]:
USELESS_COLUMNS = ['FLAG_DOCUMENT_10',
                   'FLAG_DOCUMENT_12',
                   'FLAG_DOCUMENT_13',
                   'FLAG_DOCUMENT_14',
                   'FLAG_DOCUMENT_15',
                   'FLAG_DOCUMENT_16',
                   'FLAG_DOCUMENT_17',
                   'FLAG_DOCUMENT_19',
                   'FLAG_DOCUMENT_2',
                   'FLAG_DOCUMENT_20',
                   'FLAG_DOCUMENT_21']

In [9]:
df= df.drop(USELESS_COLUMNS,axis=1)

In [10]:
df.shape

(356255, 1130)

In [11]:
import xgboost as xgb
from xgboost import plot_importance

/Users/shuozhang/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
print("Starting xgb. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

Starting xgb. Train shape: (307511, 1130), test shape: (48744, 1130)


In [13]:
del df
gc.collect()

14

In [14]:
# 测试 取100行
train_df=train_df.iloc[:100,:]
test_df=test_df.iloc[:100,:]

In [15]:
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
X=train_df[feats]
y=train_df['TARGET']

In [37]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [38]:
import pandas as pd
import numpy as np
from __future__ import print_function
from __future__ import division
import xgboost as xgb
from sklearn.cross_validation import cross_val_score
from bayes_opt import bayesian_optimization
import sklearn.cross_validation as cv

In [ ]:
def xgboostcv(max_depth,
              gamma,
              min_child_weight,
              colsample_bylevel,
              subsample,
              colsample_bytree,
              reg_lambda,
              reg_alpha,
              silent=True,
              objective='binary:logistic',
              learning_rate=0.02,
              n_estimators=5000):
    return cross_val_score(xgb.XGBClassifier(max_depth=int(max_depth),
                                             learning_rate=learning_rate,
                                             n_estimators=n_estimators,
                                             colsample_bylevel=colsample_bylevel,
                                             silent=silent,
                                            objective=objective,
                                            gamma=gamma,
                                            min_child_weight=min_child_weight,
                                            subsample=subsample,
                                            colsample_bytree=colsample_bytree,
                                            reg_alpha=reg_alpha,
                                            reg_lambda=reg_lambda),
                           X,
                           y,
                           'roc_auc',
                           cv=5).mean()

if __name__ == "__main__":
    
    xgboostBO = bayesian_optimization.BayesianOptimization(xgboostcv,
                                 {'gamma':(0.01,1),
                                 'max_depth' : (4,20),
                                 'min_child_weight' : (2,10),
                                 'subsample' : (0.4,0.9),
                                 'colsample_bytree' : (0.4,0.9),
                                 'colsample_bylevel' : (0.7,1),
                                 'reg_alpha' : (0,0.01),
                                 'reg_lambda' : (0,0.01)})
    xgboostBO.maximize(init_points=2, n_iter = 2)
    print('-'*53)
    print('Final Results')
    print('XGBOOST: %f' % xgboostBO.res['max']['max_val'])

/Users/shuozhang/anaconda/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GaussianProcess is deprecated; GaussianProcess was deprecated in version 0.18 and will be removed in 0.20. Use the GaussianProcessRegressor instead.
  warnings.warn(msg, category=DeprecationWarning)


Initialization
------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bylevel |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   reg_alpha |   reg_lambda |   subsample | 
    1 | 01m32s |    0.50000 |              0.7674 |             0.6418 |    0.9690 |     13.3932 |             9.7846 |      0.0025 |       0.0025 |      0.7896 | 


In [ ]:
folds = KFold(n_splits= 5, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
dtest=xgb.DMatrix(test_df[feats])
    
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
    dtrain = xgb.DMatrix(train_df[feats].iloc[train_idx],train_df['TARGET'].iloc[train_idx])
    dvalid = xgb.DMatrix(train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx])
    valid_y=train_df['TARGET'].iloc[valid_idx]

       # xgb
    params = {'eval_metric': 'auc',
              'objective': 'binary:logistic',
              'booster':'gbtree',
              'tree_method': 'auto',
              'nthread' : 4,
              'eta' : 0.02,
               'max_leaves': 40,
              'max_depth' : 16,
              'max_bin': 255,
              'min_child_weight' : 4,
              'subsample' : 0.5,
              'colsample_bytree' : 0.5,
              'colsample_bylevel' : 1,
              'alpha' : 0.001,
              'lambda' : 0.001,
              'scale_pos_weight': 1}
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
        
    model=xgb.train(params, dtrain, 5000, watchlist, maximize=True, early_stopping_rounds = 100, verbose_eval=100)
    oof_preds[valid_idx] = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
    sub_preds += model.predict(dtest,ntree_limit=model.best_ntree_limit) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df = pd.DataFrame(model.get_fscore().items(), columns=['feature','importance']).sort_values('importance', ascending=False)
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
    del model, dtrain, dvalid, valid_y

print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
test_df['TARGET'] = sub_preds
test_df[['SK_ID_CURR', 'TARGET']].to_csv('xgb_1000feature.csv', index= False)

[0]	train-auc:0.741412	valid-auc:0.602773
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 100 rounds.
[100]	train-auc:0.957363	valid-auc:0.76541
[200]	train-auc:0.985686	valid-auc:0.774812
[300]	train-auc:0.995787	valid-auc:0.781254
[400]	train-auc:0.998805	valid-auc:0.784253
[500]	train-auc:0.999628	valid-auc:0.785623
[600]	train-auc:0.99989	valid-auc:0.78576
[700]	train-auc:0.999974	valid-auc:0.785788
Stopping. Best iteration:
[649]	train-auc:0.999944	valid-auc:0.785963

Fold  1 AUC : 0.785963


In [ ]:
cols = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]
plt.figure(figsize=(8, 10))
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
plt.title('XGB Features (avg over folds)')
plt.tight_layout
plt.show()

In [ ]:
feature_importance_df.to_csv('feature_importance_xgb1000features.csv', index=False)